In [71]:
from __future__ import annotations

import re
from typing import Iterable, Pattern, Text




def _isplit(
    text: bytes, pattern: Pattern[bytes], include_separators: bool = False
) -> Iterable[bytes]:

    prev_end = 0
    for separator in re.finditer(pattern, text):
        # Yield the text before separator.
        yield text[prev_end : separator.start()]  # noqa: E203

        # Yield separator.
        if include_separators and (piece := separator.group(0)):
            yield piece

        # Update the start position.
        prev_end = separator.end()

    # Yield the text after the last separator.
    yield text[prev_end:]


class _CustomText(Text):
    """A custom string type for subclassing."""

    def __repr__(self) -> Text:
        """Return a string representation of the object."""
        return f"{self.__class__.__name__}({super().__repr__()})"


class Ansi():

    PATTERN = re.compile(br"(\x1B\[[\d;]*[a-zA-Z])")

    def __init__(self, input) -> None:
        self.input = input
        
    def escapes(self) -> Iterable[Text]:
        """Yield ANSI escapes and text in the order they appear."""
        for match in _isplit(self.input, self.PATTERN, include_separators=True):
            if not match:
                continue

            yield match 
            
    def message(self) -> Iterable[Text]:
        """Yield ANSI text in the order they appear."""
        msg = b''
        for match in _isplit(self.input, self.PATTERN, include_separators=False):
            if not match:
                continue

            msg += match             
        
        return msg    

MSG_TYPE_CMD        = 0
MSG_TYPE_OPENTHREAD = 1
MSG_TYPE_MATTER     = 2

class MessageClassify():
    
    def __init__(self, input:bytes) -> None:
        self.text = Ansi(input).message()    
        self.type = self._classify()

    def _classify(self):
        OT_PATTERN = re.compile(b"OPENTHREAD\:\[[a-zA-Z]\]")
        
        if OT_PATTERN.search(self.text):
            return MSG_TYPE_OPENTHREAD
        else:
            return MSG_TYPE_CMD
            
    def msg_type(self):
        return self.type

    def msg_text(self):
        return self.text

           
input = b'\x1b[0;32mI(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\x1b[0m\r\n'            
text = Ansi(input)
list(text.escapes())

print(text.message())

msg = MessageClassify(input)
    
print(msg.msg_text())
print(msg.msg_type())

b'I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\r\n'
b'I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\r\n'
1


In [36]:
import re
from typing import Iterable, Pattern


input = b'\x1b[0;32mI(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\x1b[0m\r\n'

     
def ansi_split(
    text: bytes, pattern: Pattern[bytes], include_separators: bool = False
) -> Iterable[bytes]:
    prev_end = 0
    index = 0
    for separator in re.finditer(pattern, text):
        
        index += 1
        print(f'index {index}, start{ separator.start() }')
        print(separator)
        print('*' * 32)
        # Yield the text before separator.
        yield text[prev_end : separator.start()]  # noqa: E203

        # Yield separator.
        if include_separators and (piece := separator.group(0)):
            yield piece

        # Update the start position.
        prev_end = separator.end()

    # Yield the text after the last separator.
    yield text[prev_end:]

def ansi_escap(input):
    PATTERN = re.compile(br"(\x1B\[[\d;]*[a-zA-Z])")    
    for match in ansi_split(input, PATTERN):
        print(match)
        if not match:
            continue

        yield match  

# test = list(ansi_escap(input))  
# print(test)
print('test 1')
rslt = b''.join(ansi_escap(input))       
print(rslt)


print('test 2')
input_2 = b'I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\r\n'

rslt = b''.join(ansi_escap(input_2))     
print(rslt)


test 1
index 1, start0
<re.Match object; span=(0, 7), match=b'\x1b'>
********************************
b''
index 2, start94
<re.Match object; span=(94, 98), match=b'\x1b'>
********************************
b'I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788'
b'\r\n'
b'I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\r\n'
test 2
b'I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\r\n'
b'I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\r\n'


In [68]:

# 在Python中，`re.compile()`函数用于编译正则表达式，生成一个正则表达式对象。这个对象可以用于匹配和查找字符串。
# 如果你想要传入一个`bytes`类型的字符串，你就需要使用`bytes`类型的正则表达式。
# 以下是一个例子：

import re

# define a bytes string
bytes_string = b" OPENTHREAD:[W] Hello, World!"

# define a bytes regex
bytes_regex = re.compile(b"OPENTHREAD\:\[[a-zA-Z]\]")

# use the regex to search the bytes string
match = bytes_regex.search(bytes_string)

# print the result
print(match)
if match:
    print("Match found!")
else:
    print("No match found.")


<re.Match object; span=(1, 15), match=b'OPENTHREAD:[W]'>
Match found!


In [2]:
# Define a byte string
byte_string = b"hello world"

# Convert the byte string to a string using the decode() method
decoded_string = byte_string.decode("utf-8")

# Print the decoded string
print(decoded_string)

input_2 = b'I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788\r\n'
print(input_2.decode('utf-8'))

hello world
I(592720) OPENTHREAD:[I] MeshForwarder-:     src:[fe80:0:0:0:74a9:dac6:5296:64b8]:19788



In [3]:
keybuf= b'abcdefg'

print(keybuf)

keybuf = keybuf[:-1]

print(keybuf)

b'abcdefg'
b'abcdef'


In [9]:
import curses



curses.initscr()

print(curses.KEY_BACKSPACE)

key_name = curses.keyname(curses.KEY_BACKSPACE)
print(key_name)

key_name = curses.keyname(127)
print(key_name)

print(curses.KEY_DC)

263
b'KEY_BACKSPACE'
b'^?'
